### Part 1. The data

We'll use a Markov Chain model to build a sentence using words in the Science Questions dataset.

Sentences are generated by picking a starting word and repeatedly choosing a random next word (based on transitions learned from the data) until a sentence is complete.

For example if we had two questions such as *'What is love?'* and *'What the heck?'*, our model would say that **What** can be followed by either **is** or **love**.

The goal for the data is to generate the transitions. We'll represent this output as ```dict```s, where ```key, value``` pairs are a word and list of next words, e.g.:

```python
{ 'What' : ['is', 'the'],
  'is' : ['love'],
  # and so on
}
```

Markov chain [explainer](https://setosa.io/ev/markov-chains/)


In [0]:
import pandas as pd
import csv
from collections import defaultdict, Counter
from itertools import accumulate
import json
import re
from typing import List, DefaultDict
import random

In [0]:
# Data path variables
PATH = '/content/drive/My Drive/datasets/science_questions'

In [0]:
# Read the data
with open(f'{PATH}/questions.csv') as f:
    reader = csv.DictReader(f)
    raw_questions = [row['question'] for row in reader]

In [16]:
# Example output
raw_questions[0:10]

['Which property of an object is identified using the sense of smell? (A) color (B) odor (C) temperature (D) weight',
 'Which type of energy is needed to cut a piece of wood into smaller pieces with a saw? (A) light (B) heat (C) sound (D) mechanical',
 'Which material is the best conductor of electricity? (A) metal (B) glass (C) wood (D) plastic',
 'What force causes objects to be pulled toward Earth? (A) friction (B) gravity (C) light (D) electricity',
 'When a rock is placed in a graduated cylinder containing water, the height of the water will (A) decrease (B) increase (C) remain the same',
 'Which energy transformation occurs when a person hits a drum with a drumstick? (A) electrical to light (B) sound to electrical (C) light to mechanical (D) mechanical to sound',
 'A magnet and a metal paper clip have the strongest magnetic attraction when the distance between them is (A) 4 centimeters (B) 8 centimeters (C) 12 centimeters (D) 16 centimeters',
 'A student rubs her hands together o

By looking at the data we'll want *two* Markov chain models since questions and answers look quite different from one another. 

We'll generate two sets:
> 1. Question transitions
2. Answer transitions




In [17]:
# Use regex to separate questions from answers
# This looks for the letters A-D in parentheses, and splits the string where it finds them.
# See notes below
re.split("\([A-D]\)", raw_questions[0])


['Which property of an object is identified using the sense of smell? ',
 ' color ',
 ' odor ',
 ' temperature ',
 ' weight']

**Regex notes**
>```+```</br>
Causes the resulting RE to match 1 or more repetitions of the preceding RE. ab+ will match ‘a’ followed by any non-zero number of ‘b’s; it will not match just ‘a’.

>```[ ]``` </br>
Used to indicate a set of characters. In a set: Characters can be listed individually, e.g. ```[amk]``` will match 'a', 'm', or 'k'. Ranges of characters can be indicated by giving two characters and separating them by a '-', for example ```[a-z]``` will match any lowercase ASCII letter, ```[0-5][0-9]``` will match all the two-digits numbers from 00 to 59, and ```[0-9A-Fa-f]``` will match any hexadecimal digit. If - is escaped (e.g. ```[a\-z]```) or if it’s placed as the first or last character (e.g. ```[-a]``` or ```[a-]```), it will match a literal '


>```\``` </br>
Either escapes special characters (permitting you to match characters like '*', '?', and so forth), or signals a special sequence; special sequences are discussed below.

>```(...)``` </br>
Matches whatever regular expression is inside the parentheses, and indicates the start and end of a group; the contents of a group can be retrieved after a match has been performed, and can be matched later in the string with the \number special sequence, described below. To match the literals ```'('``` or ```')'```, use ```\(``` or ```\)```, or enclose them inside a character class: ```[(]```, ```[)]```.

> ```\s``` </br>
For Unicode ```(str)``` patterns:
Matches Unicode whitespace characters (which includes ```[ \t\n\r\f\v]```, and also many other characters, for example the non-breaking spaces mandated by typography rules in many languages). If the ASCII flag is used, only ```[ \t\n\r\f\v]``` is matched.

</br>

Looking at the questions data, every question has 3 to 4 answers, so the result of the split should have 4 to 5 elements (including the questions text).


In [18]:
# Check if any of the questions does not fall into the above reasoning
pattern = "\([A-D]\)"

for q in raw_questions:
    if len(re.split(pattern, q)) not in [4, 5]:
        print(q)
        break

The floor of Lucy's classroom is in the shape of a rectangle. It is 20 yards long and its area is 180 square yards. What is the width of Lucy's classroom? A. 9 yards B. 18 yards C. 50 yards D. 70 yards


In [0]:
# Build regex cases iteratively to cover all the questions
splits = [
  "\([A-D]\)",    # (A) python (B) haskell (C) javascript (D) ruby
  "\s[A-D]\.\s",  #  A. python  B. haskell  C. javascript  D. ruby
  "\s[1-4]\.\s",  #  1. python  2. haskell  3. javascript  4. ruby
  "\s[A-D]\s",    #  A  python  B  haskell  C  javascript  D  ruby
  "\s[FGHJ]\s",   #  F  python  G  haskell  H  javascript  J  ruby
  "\n [A-D]\s"    #   A python
                  #   B haskell
                  #   C javascript
                  #   D ruby
]


In [0]:
# Initialise sentinels
START = "__START__"
STOP = "__STOP__"

Sentinels will be used to add a transition from ```START``` to the first word of every sentence, and a transition from the last-est word of every sentence to ```STOP```.

Logic:

```python
word = random_word_after(START)
while word != STOP:
  yield word
  word = random_word_after(word)
```

The strategy is as follows:

- Collect the question *sentences* and answer *sentences* separately
- Use the sentences to generate transitions ```dict```s
- Serialise teh transionts, so they cna be used in other programs

In [22]:
# Collecting

questions = []
answers = []

for q in raw_questions:
    for pattern in splits:
        pieces = [x.strip() for x in re.split(pattern, q)]
        # If we collected 4 to 5 pieces
        if len(pieces) in [4, 5]:
            # Add the first to the questions and the rest to the answers
            questions.append(pieces[0])
            answers.extend(pieces[1:])
            # Move to the next questions
            break
    else:
        # Executed only when the loop is not terminated by break
        # This finds elements that the if does not catch
        print(q)

# More on for...else loops here
# https://www.geeksforgeeks.org/using-else-conditional-statement-with-for-loop-in-python/


Tyrone taped a list of things to do on his mirror so he would not forget to do them. The list included the following tasks: 
 
 • Turn off the water while brushing your teeth. 
 • Turn off the lights when you leave a room. 
 • Close the refrigerator door quickly. 
 • Do not leave the outside doors open. 
 • Only take as much food as you can eat. 
  
 Based on Tyrone’s list, what is he trying to accomplish by performing these tasks?
After one minute, which item is most likely too hot to touch?


With the list of **questions** and **answers** we can create the transitions ```dict```s:

In [0]:
"""
Regex to match
    - a period
    - a comma
    - a question mark
    - a word containing non of the above or spaces
"""
regex = "[^ ?\.,]+|\?|\.|\,"


def make_transitions(sentences: List[str]) -> DefaultDict[str, str]:
    """
    Takes a list of sentences and:
        - Splits the words in each sentence
        - Enters each word as a dictionary key
        - Appends subsequent words to a list for each key

    Each word in a list appears with higher / lower frequency
    according to how often is found in the sentences.

    This allows random choice to chain the words with higher probability
    to follow a prvious one as they have higher frequency on the list.
    """
    transitions = defaultdict(list)

    for sentence in sentences:
        words = [START] + re.findall(regex, sentence) + [STOP]
        for prev_word, next_word in zip(words, words[1:]):
            transitions[prev_word].append(next_word)
    
    return transitions

In [0]:
# Make the transitions
q_transitions = make_transitions(questions)
a_transitions = make_transitions(answers)



In [56]:
# Inspect the first 5 starting words
q_transitions['__START__'][0:5]

['Which', 'Which', 'Which', 'What', 'When']

In [0]:
# Sentence generators
def next_word(transitions: DefaultDict[str, str], key:str) -> str:
    """
    Picks a word from the list correspondy to key in the transitions dict.
    """ 
    return random.choice(transitions.get(key, [STOP]))

def markov_gen(transitions: DefaultDict[str, str]) -> str:
    """
    Starting at the START sentinel, calls next_word to
    pick the word to use as a key to build a sentence.

    Continues until STOP is encountered.
    """
    # Get the next word starting at the START sentinel
    word = next_word(transitions, START)
    # Continue until word == STOP sentinel
    while word != STOP:
      yield word
      word = next_word(transitions, word)

In [37]:
' '.join(markov_gen(q_transitions))


'Crows are composed of humans ?'

In [0]:
# Svae transition as JSON
with open('questions.json', 'w') as f:
    f.write(json.dumps(q_transitions))

with open('answers.json', 'w') as f:
    f.write(json.dumps(a_transitions))

### Optimisation

 ~ - For each list of words:
1. Count occurrence with Counter()
2. Create set and set length of uniques
3. Create empty array of len(set), with the specific constructor
4. Based on len set assign words
> e.g. if I have 10 unique words and counter has 20 of one, add two to the array
5. Final array should represent weights scaled to the number of uniques
> a random selection should have more or less probability based to what was found by counter in the data ~
**Not needed**





In [0]:
def compress_transitions(transitions):
    compressed = {}
    for token, next_tokens in transitions.items():
        counts = Counter(next_tokens)
        compressed[token] = list(zip(counts.keys(), accumulate(counts.values())))
    return compressed

In [0]:
with open('questions_compressed.json', 'w') as f:
    f.write(json.dumps(compress_transitions(q_transitions)))

In [0]:
# Let's try to optimise giving a frequencey to each word
counts = Counter(q_transitions['__START__'])

In [51]:
totals = []
for key, value in counts.items():
    totals.append(value)

number_of_words = sum(totals)

number_of_words

2000

In [53]:
uniques = set()
for key, value in counts.items():
    if key not in uniques:
        uniques.add(key)

len(uniques)


229

In [54]:
uniques

{'13',
 '22',
 'A',
 'About',
 'Above',
 'Adaptation',
 'Adaptations',
 'After',
 'Air',
 'Alex',
 'Algae',
 'All',
 'Although',
 'Aluminum',
 'An',
 'Andy',
 'Animal',
 'Animals',
 'Anita',
 'Annette',
 'Another',
 'Apple',
 'Approximately',
 'April',
 'As',
 'Ashley',
 'At',
 'Baby',
 'Baking',
 'Barry',
 'Base',
 'Based',
 'Beans',
 'Bears',
 'Before',
 'Benjamin',
 'Beryl',
 'Bill',
 'Both',
 'Butterflies',
 'Cacti',
 'Cameron',
 'Camouflage',
 'Carolina',
 'Changes',
 'Charlie',
 'Chemical',
 'Chris',
 'Christy',
 'Chromosomes',
 'Clouds',
 'Coal',
 'Competition',
 'Copper',
 'Coral',
 'Corals',
 'Cows',
 'Crows',
 'Darker',
 'David',
 'Decomposers',
 'Delilah',
 'Derek',
 'Different',
 'Directions',
 'Ducks',
 'During',
 'Early',
 'Earth',
 'Earthworms',
 'Electricity',
 'Elliot',
 'Eric',
 'Erin',
 'Erosion',
 'Eveline',
 'Every',
 'Everything',
 'Eye',
 'Farmers',
 'Felicia',
 'Females',
 'Fifth',
 'Flexibility',
 'Food',
 'For',
 'Fossils',
 'Four',
 'Fourth',
 'Francis',
 'Fr